# An exploration of the relationship between intelligence and learning

The relationship between brain structure and function has long been a central focus of neuroscientific endeavors. The dataset analyzed in this study consists of features that have been used to assess the ways in which brain size may relate to intelligence. In order to build on this investigation, we would like to examine how intelligence may related to learning. Furthermore, we aim to explore which particular intelligence features might have the greatest impact on learning.

## Load dataset
The dataset includes 40 participants with Full Scale Intelligent Quotient (FSIQ), Verbal Intelligent Quotient (VIQ), and Performance Intelligent Quotient (PIQ) scores. The brain size of each individual was determined through magnetic resonance imaging (MRI). Height and weight for each participant, and gender was specified.

Use pandas to load brain.csv data.

In [4]:
import pandas as pd

csvfile = ('/Users/Storme/brainhack/Alley-S-QLSC612/Original_Data/brainsize.csv')

data = pd.read_csv(csvfile, sep=';', na_values='.', index_col=0)
data.head()

,Gender,FSIQ,VIQ,PIQ,Weight,Height,MRI_Count
1,Female,133,132,124,118.0,64.5,816932
2,Male,140,150,124,NaN,72.5,1001121
3,Male,139,123,150,143.0,73.3,1038437
4,Male,133,129,128,172.0,68.8,965353
5,Female,137,132,134,147.0,65.0,951545
